In [44]:
import os
import sys
import pandas as pd
import numpy as np
from openpyxl import load_workbook
import copy


################# Clean_output Function ###################
def clear_output():                                       #
                                                          #
  os.system('cls' if os.name == 'nt' else 'clear')        #
  if is_in_notebook():                                    #
    from IPython.display import clear_output as clear     #
    clear()                                               #
                                                          #
def is_in_notebook():                                     #
  return 'ipykernel' in sys.modules                       #
###########################################################

#### Function 1: Check Inventory and Trucks  
def Function1(loc):
    clear_output()
    try:
        wb = load_workbook(loc)
        print('File opened successfully!')
    except:
        sys.exit('Unable to open the file.')
    ws = wb['DATA']    
    ws.cell(row = 1, column = 7).value = "Number of Truck"
    i = 2 
    j = 1
    while ws.cell(row = i, column = j).value != None:
        ws.cell(row = i, column = j).value = i - 1
        i = i + 1    
    wb.save(loc)
    
    df = pd.read_excel("C:\\Users\Cuiziming\Desktop\project.xlsx", dtype = 'str')
    df['Number to Ship Today'].fillna("-", inplace=True)
    df['Truck Capacities'].fillna("-", inplace=True)
    df['Number of Truck'].fillna("-", inplace=True)
    print(df.iloc[:, [0, 1, 2, 5, 6]])    
    return 1

#### Function 2: Truck Arrangement
def Function2(loc):
    clear_output()
    try:
        wb = load_workbook(loc)
        print('File opened successfully!')
    except:
        sys.exit('Unable to open the file.')
    ws = wb['DATA']

    ws.cell(row = 1, column = 7).value = "Number of Truck"
    i = 2 
    j = 1
    while ws.cell(row = i, column = j).value != None:
        ws.cell(row = i, column = j).value = i - 1
        i = i + 1    

    print("Please enter the number of trucks you want to schedule today (0~6):")
    Truck_Max = int(input())
    Sum_truck = 0
    m = 2
    n = 6
    while Sum_truck < Truck_Max:
        print("Please enter the capacity of truck you would like to schedule:")
        Capacity = input()
        ws.cell(row = m, column = n).value = Capacity
        print("Please enter the number of such trucks you would like to arrange:")
        Each_truck = int(input())
        ws.cell(row = m, column = n + 1).value = Each_truck
        Sum_truck = Sum_truck + Each_truck
        m = m + 1 
    wb.save(loc)
    print("Truck Arrangement success! Please enter function 1 to view the result!")
    return 2

#### Function 3:
def Function3(loc):
    clear_output()
    df = pd.read_excel("C:\\Users\Cuiziming\Desktop\project.xlsx", dtype = 'str')
    df['Number to Ship Today'].fillna("-", inplace=True)
    df['Truck Capacities'].fillna("-", inplace=True)
    df['Number of Truck'].fillna("-", inplace=True)
    print(df.iloc[:, [0, 1, 2, 5, 6]]) 
    try:
        wb = load_workbook(loc)
        print('File opened successfully!')
    except:
        sys.exit('Unable to open the file.')
    ws = wb['DATA']    
    ws.cell(row = 1, column = 7).value = "Number of Truck"
    i = 2 
    j = 1
    while ws.cell(row = i, column = j).value != None:
        ws.cell(row = i, column = j).value = i - 1
        i = i + 1
    print("Do you want to add items, Y or N?")
    n = 3
    Y_N = input()
    while Y_N == "Y":
        print("Please enter the item number(1-50) you want to add!")
        item_NO = int(input())
        m = item_NO + 1
        print("How many such item do you want to arrange?")
        item_num = int(input())
        ws.cell(row = m, column = n).value = item_num
        print("Do you continue to add items, Y or N?")
        Y_N = input()
    wb.save(loc)
    return 3

#### Function 4: Ship Items
def find_truck(List_TruckRest):
    length_TruckRest = len(List_TruckRest)   
    truck_point = 0
    End_Arrange = False
    New_Trucks = None
    #List_ItemRest = copy.deepcopy(List_Item)
    while End_Arrange == False and truck_point <= length_TruckRest - 1:
        if int(List_TruckRest[truck_point][1]) > 0:
            New_Trucks = copy.deepcopy(List_TruckRest[truck_point])
            List_TruckRest[truck_point][1] = List_TruckRest[truck_point][1] - 1
            New_Trucks[1] = 1
            #print("New_truck = ",New_Truck)
            End_Arrange = True
        elif truck_point > length_TruckRest - 1:
            End_Arrange = True
            New_Trucks = None
        else:
            truck_point = truck_point + 1           
    return New_Trucks     

def find_item(New_Truck, List_ItemRest):
    item_point = 0
    New_Item = None
    End_Arrange = False
    length_itemRest = len(List_ItemRest)
    while End_Arrange == False and item_point <= length_itemRest - 1:
        if New_Truck != None and New_Truck[0] >= List_ItemRest[item_point][0] and List_ItemRest[item_point][1] > 0:
            
            New_Item = List_ItemRest[item_point].copy()
            New_Item[1] = 1
            List_ItemRest[item_point][1] = List_ItemRest[item_point][1] - 1
            #print("New_item = ",New_Item)
            End_Arrange = True
        elif item_point > length_itemRest - 1:
            New_Item = None
            End_Arrange = True
            #print("Ship over!")
        else:        
            item_point = item_point + 1
    
    #print(New_Item)
    return New_Item

#### RestSpace:
def Rest_space(Rest_TruckSpace, Ship_List, Ship_Num, New_Item, List_ItemRest):
    
    Ship_List[Ship_Num] = Ship_List[Ship_Num] + [[New_Item[0],New_Item[1]]]
    
    New_Item = find_item(Rest_TruckSpace, List_ItemRest)
    while New_Item != None:
        Ship_List[Ship_Num] = Ship_List[Ship_Num] + [[New_Item[0],New_Item[1]]]
        Rest_TruckSpace[0] = Rest_TruckSpace[0] - New_Item[0]
        New_Item = find_item(Rest_TruckSpace, List_ItemRest)
    return  


#### 3. Set current order plan 
def Function4(loc):
    
    try:
        wb = load_workbook(loc)
        print('File opened successfully!')
    except:
        sys.exit('Unable to open the file.')
    ws = wb['DATA']

    ws.cell(row = 1, column = 7).value = "Number of Truck"
    i = 2 
    j = 1
    while ws.cell(row = i, column = j).value != None:
        ws.cell(row = i, column = j).value = i - 1
        i = i + 1

#### Item
    List_Item = []
    m = 2
    n = 3
    while m <= 51:
        if ws.cell(row = m, column = n).value != None:
            Item_num = int(ws.cell(row = m, column = n).value)
            Item_weight = int(ws.cell(row = m, column = n - 1).value)
            Item_NO = int(ws.cell(row = m, column = n - 2).value)
            List_Item = List_Item + [[Item_weight, Item_num, Item_NO]]
        m = m + 1

    
#### Truck    
    List_Truck = []
    p = 2
    q = 6
    while p <= 7:
        if ws.cell(row = p, column = q).value != None:
            Truck_weight = int(ws.cell(row = p, column = q).value)
            Truck_num = int(ws.cell(row = p, column = q + 1).value)
            List_Truck = List_Truck + [[Truck_weight, Truck_num]]
        p = p + 1    

    wb.save(loc)
    
#### Ship Plan
    List_Item.sort(reverse=True)
    List_Truck.sort(reverse=True)
    List_ItemRest = copy.deepcopy(List_Item)
    List_TruckRest = copy.deepcopy(List_Truck)
            
    
#### Choose max capacity truck    
    Ship_List = []    
    New_Truck = find_truck(List_TruckRest)    
#### Choose max weight item
    if New_Truck != None:        
        New_Item = find_item(New_Truck, List_ItemRest)
        Ship_List = Ship_List + [[New_Truck]]
    else:
        New_Item = None
#### Ship        
    while New_Truck != None and New_Item != None:
        Rest_TruckSpace = [(New_Truck[0] - New_Item[0]), 1]
        Ship_Num = len(Ship_List) - 1
        ###### RestSPace deal with
        Rest_space(Rest_TruckSpace, Ship_List, Ship_Num, New_Item, List_ItemRest)
#### Choose max capacity truck      
        #print(List_TruckRest)
        New_Truck = find_truck(List_TruckRest)    
#### Choose max weight item
        if New_Truck != None:        
            New_Item = find_item(New_Truck, List_ItemRest)
            if New_Item != None:
                Ship_List = Ship_List + [[New_Truck]]
            else:
                return_Truck = 0
                while return_Truck <= len(List_TruckRest) - 1:
                    if List_TruckRest[return_Truck][0] == New_Truck[0]:
                        List_TruckRest[return_Truck][1] = List_TruckRest[return_Truck][1] + 1
                        return_Truck = len(List_TruckRest)    
                    return_Truck = return_Truck + 1    
                    
    print("############################################################################################################################")
    print("Number of trucks before transport:")
    print(List_Truck)
    print("Quantity of goods before shipment:")
    length_1 = 0
    while length_1 <= len(List_Item) - 1:
        print("Item           Weight            Number")
        print(List_Item[length_1][2],"              ",List_Item[length_1][0],"            ",List_Item[length_1][1])
        length_1 = length_1 + 1
    print(List_Item)
    print("############################################################################################################################")
    print("The rest of Trucks:")
    print(List_TruckRest)
    print("The rest of Items:")
    print(List_ItemRest)
    print("############################################################################################################################")
    print("Shipping plan:")
    length_2 = 0
    while length_2 <= len(Ship_List) - 1:
        print(Ship_List[length_2])
        length_2 = length_2 + 1
    #print(Ship_List)
    print("############################################################################################################################")

    return  

#### Function 5: Clear form
def Function5(loc):
    clear_output()
    try:
        wb = load_workbook(loc)
        print('File opened successfully!')
    except:
        sys.exit('Unable to open the file.')
    ws = wb['DATA']
    
    ws.cell(row = 1, column = 7).value = "Number of Truck"
    ws.delete_cols(7)
    ws.delete_cols(6)
    ws.delete_cols(3)

    ws.cell(row = 1, column = 3).value = "Number to Ship Today"
    ws.cell(row = 1, column = 6).value = "Truck Capacities"
    ws.cell(row = 1, column = 7).value = "Number of Truck"

    i = 2 
    j = 1
    while ws.cell(row = i, column = j).value != None:
        ws.cell(row = i, column = j).value = i - 1
        i = i + 1    
    wb.save(loc)
    print("Clear success! Please enter function 1 to view the result!")
    return 5
    
    
#### Function 6: Exit    
def Fuction6():
    return 6

#### Main Menu
def Menu1():
    clear_output()
    print("############################################################################################################################")
    print("#                                                                                                                          #")
    print("#                                                                                                                          #")
    print("#                             Welcome to MutiSales Distribution System                                                     #")
    print("#                                                                                                                          #")
    print("#                                                                                                                          #")
    print("#                                                                                                                          #")
    print("#                             1. Check Inventory and Trucks                                                                #")
    print("#                             2. Truck Arrangement                                                                         #")
    print("#                             3. Set current order plan                                                                    #")
    print("#                             4. Ship items                                                                                #")
    print("#                             5. Clear form                                                                                #")
    print("#                             6. Exit                                                                                      #")
    print("#                                                                                                                          #")
    print("#                             Please input the function number(1-6) you want to execute and press Enter:                   #")
    print("#                                                                                                                          #")
    print("#                                                                                                                          #")
    print("#                                                                                                                          #")
    print("############################################################################################################################")
    Function_num = int(input("Enter Function "))
    print("############################################################################################################################")
    return Function_num

######## Main function

loc = "C:\\Users\Cuiziming\Desktop\project.xlsx"
Function_num = Menu1()
while Function_num != 6:
#### Function1
    if Function_num == 1:
        Function1(loc)
        print("")
        print("If you want to return to Main Menu, please press enter!")
        input()
        Function_num = Menu1()
#### Function2
    elif Function_num == 2:
        Function2(loc)
        print("")
        print("If you want to return to Main Menu, please press enter!")
        input()
        Function_num = Menu1()        
#### Function3
    elif Function_num == 3:
        Function3(loc)
        print("")
        print("If you want to return to Main Menu, please press enter!")
        input()
        Function_num = Menu1()
#### Fuction4
    elif Function_num == 4:
        Function4(loc)
        print("")
        print("If you want to return to Main Menu, please press enter!")
        input()
        Function_num = Menu1()
#### Function5    
    elif Function_num == 5:
        Function5(loc)
        print("")
        print("If you want to return to Main Menu, please press enter!")
        input()
        Function_num = Menu1()
#### Function6
    elif Function_num == 6:
        Function_num = Fuction6()            
#### Other status
    else:
        Function_num = Menu1()

print("Exit System!")







############################################################################################################################
#                                                                                                                          #
#                                                                                                                          #
#                             Welcome to MutiSales Distribution System                                                     #
#                                                                                                                          #
#                                                                                                                          #
#                                                                                                                          #
#                             1. Check Inventory and Trucks                                                                #
